<a href="https://colab.research.google.com/github/KnoBuddy/Aviato/blob/main/Erlich_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KnoBuddy's Colab Notebook Implementation of Erlich a text2image generator for logos

# Connect to Google Drive and Create Repo

In [31]:
ls

'=1.0.8'                  encoders/                    predict_util.py
'=2.0.0'                  erlich/                      README.md
 aesthetic_clip_embeds/   erlich.pt                    requirements.txt
 aesthetic-predictor/     guided_diffusion/            sample.py
 autoedit_api.py          guided_diffusion.egg-info/   scripts/
 autoedit.py              kl-f8.pt                     setup.py
 bert.pt                  latent-diffusion/            src/
 clip_custom/             ldm/                         taming-transformers/
 cog_predict.py           LICENSE                      wandb/
 cog.yaml                 new_folder/


Connect your google drive and create a folder name test_nb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir test_2

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 10 20:38:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# VSCode Integration (Optional WIP)

In [ ]:
! pip install colabcode

In [2]:
from colabcode import ColabCode

In [ ]:
ColabCode(port=10000,mount_drive=True)

# Clone Repo, Install Dependencies, and Init Settings

In [5]:
%cd drive/MyDrive/test_2/

/content/drive/MyDrive/test_2


In [14]:
! git lfs install

Updated git hooks.
Git LFS initialized.


In [6]:
! ls

ldm-finetune


In [7]:
! git clone https://github.com/LAION-AI/ldm-finetune

Cloning into 'ldm-finetune'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 292 (delta 134), reused 265 (delta 107), pack-reused 0
Receiving objects: 100% (292/292), 1.95 MiB | 10.54 MiB/s, done.
Resolving deltas: 100% (134/134), done.


Use this to update

In [8]:
! git pull

Already up to date.


In [7]:
%cd ldm-finetune/

/content/drive/MyDrive/test_2/ldm-finetune


In [13]:
! wandb login 02a7247d36c07136ba7b90e08d6201ee8e67bc1b

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
! git pull

In [14]:
! pip install git+https://github.com/PyTorchLightning/pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-1fy_eoyf
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-1fy_eoyf
ERROR: Operation cancelled by user


In [12]:
! pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
! git clone https://huggingface.co/laion/erlich

Cloning into 'erlich'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.

Exiting because of "interrupt" signal.
You can inspect what was checked out with 'git status'
and retry the checkout with 'git checkout -f HEAD'

^C


In [9]:
! git lfs help smudge

Sorry, no usage text found for "git-lfs"


In [27]:
! git clone https://github.com/LAION-AI/aesthetic-predictor

Cloning into 'aesthetic-predictor'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 67 (delta 21), reused 28 (delta 3), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [10]:
! git clone https://github.com/CompVis/latent-diffusion.git
! git clone https://github.com/CompVis/taming-transformers
! pip install -e ./taming-transformers

Cloning into 'latent-diffusion'...
remote: Enumerating objects: 282, done.
remote: Total 282 (delta 0), reused 0 (delta 0), pack-reused 282
Receiving objects: 100% (282/282), 28.39 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Total 1335 (delta 0), reused 0 (delta 0), pack-reused 1335
Receiving objects: 100% (1335/1335), 409.77 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (280/280), done.
Checking out files: 100% (719/719), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/test_2/ldm-finetune/taming-transformers
  Running setup.py develop for taming-transformers


In [15]:
! pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops

In [9]:
! pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/test_2/ldm-finetune
  Attempting uninstall: guided-diffusion
    Found existing installation: guided-diffusion 0.0.0
    Can't uninstall 'guided-diffusion'. No files were found to uninstall.
  Running setup.py develop for guided-diffusion


In [16]:
! pip install -r requirements.txt
! pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining latent-diffusion from git+https://github.com/CompVis/latent-diffusion.git@main#egg=latent-diffusion (from -r requirements.txt (line 23))
  Updating ./src/latent-diffusion clone (to revision main)
  Running command git fetch -q --tags
  Running command git reset --hard -q 5a6571e384f9a9b492bbfaca594a2b00cad55279
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Can't uninstall 'latent-diffusion'. No files were found to uninstall.
  Running setup.py develop for latent-diffusion
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4go_dbcl
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-4go_dbcl
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=5ecbfdf419c47926408bd3350a36f117fecde6e40bf31de90ba10fb5516d3085
  Stored in directory: /tmp/pip-ephem-wheel-cache-dkvw89xv/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [18]:
! git lfs install

Updated git hooks.
Git LFS initialized.


# Get the Models

In [ ]:
! git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
! wget -O erlich.pt https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt

In [19]:
# text encoder (required)
! wget https://dall-3.com/models/glid-3-xl/bert.pt

--2022-06-10 20:42:31--  https://dall-3.com/models/glid-3-xl/bert.pt
Resolving dall-3.com (dall-3.com)... 172.67.204.67, 104.21.74.163, 2606:4700:3035::6815:4aa3, ...
Connecting to dall-3.com (dall-3.com)|172.67.204.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2328155637 (2.2G) [application/vnd.snesdev-page-table]
Saving to: ‘bert.pt’

bert.pt             100%[===================>]   2.17G  18.0MB/s    in 2m 10s  

2022-06-10 20:44:41 (17.0 MB/s) - ‘bert.pt’ saved [2328155637/2328155637]



In [20]:
# ldm first stage (required)
! wget https://dall-3.com/models/glid-3-xl/kl-f8.pt

--2022-06-10 20:45:23--  https://dall-3.com/models/glid-3-xl/kl-f8.pt
Resolving dall-3.com (dall-3.com)... 104.21.74.163, 172.67.204.67, 2606:4700:3035::6815:4aa3, ...
Connecting to dall-3.com (dall-3.com)|104.21.74.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334732241 (319M) [application/vnd.snesdev-page-table]
Saving to: ‘kl-f8.pt’

kl-f8.pt            100%[===================>] 319.22M  25.4MB/s    in 15s     

2022-06-10 20:45:39 (20.7 MB/s) - ‘kl-f8.pt’ saved [334732241/334732241]



In [ ]:
# there are several diffusion models to choose from:

In [ ]:
# original diffusion model from CompVis
! wget https://dall-3.com/models/glid-3-xl/diffusion.pt

In [ ]:
# new model fine tuned on a cleaner dataset (will not generate watermarks, split images or blurry images)
! wget https://dall-3.com/models/glid-3-xl/finetune.pt

In [ ]:
# inpaint
! wget https://dall-3.com/models/glid-3-xl/inpaint.pt

In [21]:
# erlich
! wget -O erlich.pt https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt

--2022-06-10 20:45:53--  https://huggingface.co/laion/erlich/raw/main/model/ema_0.9999_120000.pt
Resolving huggingface.co (huggingface.co)... 18.204.39.151, 34.198.184.113, 3.208.54.125, ...
Connecting to huggingface.co (huggingface.co)|18.204.39.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135 [text/plain]
Saving to: ‘erlich.pt’

erlich.pt           100%[===================>]     135  --.-KB/s    in 0s      

2022-06-10 20:45:54 (26.4 MB/s) - ‘erlich.pt’ saved [135/135]



In [ ]:
# ongo
! wget https://huggingface.co/laion/ongo/resolve/main/ongo.pt

# Make Predictions

In [25]:
! git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
# generated images saved to ./output/
# generated image embeddings saved to ./output_npy/ as npy files

In [ ]:
# fast PLMS sampling
! python sample.py --model_path finetune.pt --batch_size 1 --num_batches 1 --text "a cyberpunk girl with a scifi neuralink device on her head"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --batch_size 1 --num_batches 12 --text "a cyberpunk girl with a scifi neuralink device on her head | trending on artstation"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --batch_size 2 --num_batches 1 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group"

In [ ]:
# sample with an init image
!python sample.py --init_image picture.jpg --skip_timesteps 10 --model_path finetune.pt --batch_size 6 --num_batches 6 --text "a cyberpunk girl with a scifi neuralink device on her head"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --guidance_scale 30.0 --seed -1 --batch_size 1 --num_batches 1 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group" --dropout 0.1 --ema_rate 0.9999 --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 32 --learn_sigma False --noise_schedule linear --num_channels 320 --num_heads 8 --num_res_blocks 2 --resblock_updown False --use_fp16 True --use_scale_shift_norm False

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path finetune.pt --guidance_scale 25.0 --seed -1 --batch_size 1 --num_batches 100 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group. the name of the group KnoBuddy is spelled out across the bottom of the logo"

In [ ]:
# classifier free guidance + CLIP guidance (better adherence to prompt, much slower)
! python sample.py --clip_guidance --model_path erlich/model/ema_0.9999_040000.pt --guidance_scale 25.0 --seed -1 --batch_size 1 --num_batches 100 --text "the logo for Kno Buddy. the style of the logo is minimalist cyberpunk vector art with psychedelic trippy vaporwave colrs. a diamond between two headphones is the icon. a diamond listening to headphones symbolizes the group. the name of the group KnoBuddy is spelled out across the bottom of the logo"

Using device: cuda
tcmalloc: large alloc 3491405824 bytes == 0x6d7c000 @  0x7fac7cf7ab6b 0x7fac7cf9a379 0x7fabc2aa750e 0x7fabc2a997c2 0x7fabfe1234c1 0x7fac780f0446 0x7fac77dc73f1 0x593784 0x594731 0x548cc1 0x51566f 0x549e0e 0x593fce 0x5118f8 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x604173
wandb: Currently logged in as: knobuddy. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /content/drive/MyDrive/test_2/ldm-finetune/wandb/run-20220610_212611-1qj6lb8o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run colorful-lake-61
wandb: ⭐️ View project at https://wandb.ai/knobuddy/ongo-eval-table
wandb: 🚀 View run at https://wandb.ai/knobuddy/ongo-eval-table/runs/1qj6lb8o
Starting run for	the lo